In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
!pip install geopandas matplotlib contextily


In [ ]:
from pathlib import Path

import shapely.wkb
import geopandas as gpd

from app.db import create_pool, get_connection_from_pool


In [ ]:
gpx_files = Path("./tmp/").glob("*.gpx")

gpx_file = next(gpx_files)

gpx_file


In [ ]:
row = gpd.read_file(gpx_file, layer="tracks", driver="GPX").iloc[0]

name = row["name"]
geometry = row["geometry"].wkt
activity = row["type"]

name, geometry, activity


In [ ]:
await create_pool()


In [ ]:
async with get_connection_from_pool() as con:
    await con.execute(
        """
        INSERT INTO tracks (name, geometry, activity)
        VALUES ($1, ST_SetSRID(ST_GeomFromText($2), 4326), $3)
        """,
        name, geometry, activity
    )


In [ ]:
async with get_connection_from_pool() as con:
    records = await con.fetch("SELECT geometry FROM tracks LIMIT 1")

record = records[0]

wkb_str = record["geometry"]
wkb = shapely.wkb.loads(wkb_str)

gdf = gpd.GeoDataFrame({"geometry": [wkb]})

gdf.plot()


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

ax = fig.add_subplot(111)

gdf.plot(ax=ax)

import io

buf = io.BytesIO()

fig.savefig(buf, format="png")

buf.seek(0)

buf.read()
